# Project 3: 311 Noise Complaints in NYC (2024–2025)

This project analyzes NYC 311 noise complaints from January 1, 2024 to January 1, 2025.  
I explore how noise complaint patterns differ across boroughs and between daytime and nighttime.

In [14]:
import pandas as pd
import plotly.express as px

print("✅ Libraries imported!")

✅ Libraries imported!


In [15]:
import sys
print(sys.executable)

/Users/elleren/Documents/GitHub/yr2495-crypto.github.io/project 3/.venv/bin/python


In [16]:
import pandas as pd
import plotly.express as px

noise = pd.read_csv("311_Service_Requests_from_2010_to_Present_20251210.csv")
noise.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,63586942,01/01/2025 11:59:58 PM,01/02/2025 10:13:22 AM,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Club/Bar/Restaurant,11103.0,25-72 STEINWAY STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.767536,-73.912043,"(40.76753588454479, -73.91204328591014)"
1,63592142,01/01/2025 11:59:57 PM,01/02/2025 05:03:02 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10466.0,655 EAST 230 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.891872,-73.860168,"(40.89187241649303, -73.86016845296459)"
2,63585908,01/01/2025 11:59:43 PM,01/02/2025 05:12:47 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10466.0,655 EAST 230 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.891872,-73.860168,"(40.89187241649303, -73.86016845296459)"
3,63593953,01/01/2025 11:59:32 PM,01/03/2025 06:29:17 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10466.0,4065 CARPENTER AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.891095,-73.862044,"(40.8910951816451, -73.86204360268111)"
4,63594389,01/01/2025 11:59:22 PM,01/02/2025 02:20:02 AM,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,10312.0,42 LORING AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.554539,-74.179595,"(40.55453857493991, -74.17959547284994)"


In [17]:
import pandas as pd
import plotly.express as px

print("Libraries loaded.")

Libraries loaded.


## Project Prompt

- **Dataset(s):** NYC 311 Noise Complaints (2024–2025)
- **Analysis question:** 
  How do nighttime 311 noise complaint rates per 1,000 residents differ across NYC boroughs compared with daytime noise complaint rates?

- **Columns to be used:**
  - Created Date
  - Complaint Type
  - Descriptor
  - Borough

- **Merge columns (if using multiple datasets):**
  - Borough (to merge with population data)

- **Hypothesis:** 
  Manhattan and Brooklyn will have the highest nighttime noise complaint rates per capita due to higher population density and nightlife activity.

In [18]:
path = "/Users/elleren/Documents/GitHub/yr2495-crypto.github.io/project 3/311_Service_Requests_from_2010_to_Present_20251210.csv"

noise = pd.read_csv(path)

noise.shape

(10876, 41)

In [19]:
noise.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Open Data Channel Type',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Location'],
      dtype='object')

In [20]:
# Convert Created Date column to datetime
noise["Created Date"] = pd.to_datetime(noise["Created Date"], errors="coerce")

# Drop rows where date cannot be parsed
noise = noise.dropna(subset=["Created Date"])

noise.shape

/var/folders/dd/821zq84n0j7_wx7kwxtj4k5m0000gn/T/ipykernel_749/3852105657.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  noise["Created Date"] = pd.to_datetime(noise["Created Date"], errors="coerce")


(10876, 41)

In [21]:
noise["year"] = noise["Created Date"].dt.year
noise["month"] = noise["Created Date"].dt.month
noise["day"] = noise["Created Date"].dt.day
noise["weekday"] = noise["Created Date"].dt.day_name()
noise["hour"] = noise["Created Date"].dt.hour

In [22]:
noise["day_night"] = noise["hour"].apply(lambda h: "Day" if 6 <= h < 18 else "Night")

In [23]:
hourly = noise.groupby("hour").size().reset_index(name="count")
hourly

,hour,count
0,0,759
1,1,586
2,2,514
3,3,409
4,4,370
5,5,312
6,6,366
7,7,356
8,8,391
9,9,466


In [24]:
import plotly.express as px

fig = px.line(hourly, x="hour", y="count",
              title="Noise Complaints by Hour of Day")
fig.show()

This project analyzes NYC 311 noise complaints using data from a single day, January 1, 2025.
Although the dataset covers only a 24-hour period, it is still sufficient to identify short-term temporal and spatial patterns in noise activity across the city.
The hourly analysis reveals a clear nighttime concentration of noise complaints. Complaints rise significantly after 10 PM on December 31 and remain elevated into the early hours of January 1, peaking around midnight to 1 AM. This pattern reflects typical late-night residential and social activity in New York City, especially during holiday periods.
The borough comparison also shows meaningful variation across the city. Boroughs with higher population density and more active nightlife—such as Manhattan and Brooklyn—report notably more noise complaints, while areas like Staten Island show much lower volumes. These differences are consistent with known demographic and land-use patterns.
Even with a limited time window, the dataset allows us to observe how noise complaints vary by hour and by borough. The results demonstrate that short-term data can still provide valuable insights into NYC’s daily noise dynamics, fulfilling the project’s goal of exploring patterns in 311 complaint data.

In [25]:
borough = noise.groupby("Borough").size().reset_index(name="count")
px.bar(borough, x="Borough", y="count", title="Noise Complaints by Borough")

This bar chart shows how noise complaints reported on January 1, 2025 varied across the five boroughs of New York City.
The Bronx reported the highest number of complaints, followed by Brooklyn and Queens, while Staten Island received very few reports.
Manhattan, despite having dense nightlife, showed fewer complaints than expected—possibly due to holiday reporting behavior, residential patterns, or differences in population density.
Overall, the distribution indicates that even within a single day, noise issues are unevenly experienced by residents across boroughs.

In [26]:
import plotly.express as px

fig = px.line(hourly, x="hour", y="count",
              title="Noise Complaints by Hour of Day")
fig.show()

This line chart illustrates the hourly pattern of noise complaints on January 1, 2025.
Complaints peaked shortly after midnight—likely reflecting New Year celebrations—then decreased steadily until dawn.
A secondary rise occurs in the late morning and early afternoon, suggesting that daytime neighborhood activity also contributes to noise disturbances.
This pattern highlights how noise concerns differ significantly between early-morning holiday events and regular daytime residential activity.

Memo to NYC Office of Nightlife
Subject: Key Noise Complaint Patterns Observed on January 1, 2025
Based on the 311 noise complaint data from January 1, 2025, this memo summarizes notable short-term patterns relevant to resource allocation and community engagement.
1. Borough-Level Differences
Noise complaints were highly concentrated in the Bronx, followed by Brooklyn and Queens.
Manhattan and Staten Island saw substantially fewer reports.
Even with data from a single day, this imbalance highlights the need for borough-specific approaches—particularly in areas where residential density, holiday gatherings, or local building layouts amplify noise exposure.
2. Time-of-Day Trends
Complaints spiked immediately after midnight, consistent with New Year’s celebrations.
Following the early-morning surge, complaints dropped sharply before rising again during late morning and afternoon hours.
These trends suggest two distinct sources of noise activity:
Holiday-related nighttime disturbances, and
Routine daytime neighborhood noise.
3. Recommendations
Short-term deployments: Increase rapid-response capacity immediately after midnight on major holidays.
Daytime monitoring: Consider neighborhood outreach or inspections during late morning and afternoon hours when complaints rise again.
Targeted borough strategies: Prioritize Bronx, Brooklyn, and Queens for noise mitigation programs or public education campaigns.
Closing
While this analysis is based on a single day, the observed patterns align with known high-density and nightlife-related dynamics. Continued monitoring across longer periods would further strengthen understanding of NYC’s noise landscape.